<br>

# Introdução


In [ ]:
import os
import ast
import json
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
#import shutil
#import urllib.request

In [ ]:
#!pip install traquitanas --upgrade

In [ ]:
import traquitanas.utils as tt

<br>

## Paths


In [ ]:
# Inicialmente faz-se necessário criar uma pasta que receberão os dados do IBGE
data_path = os.path.join('..', 'data')
input_path = os.path.join(data_path, 'input')
output_path = os.path.join(data_path, 'output')

output_path_geo = os.path.join(output_path, 'geo')
output_path_tab = os.path.join(output_path, 'tab')

os.makedirs(data_path, exist_ok=True)
os.makedirs(input_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)
os.makedirs(output_path_geo, exist_ok=True)
os.makedirs(output_path_tab, exist_ok=True)

<br>

# Dados Espaciais

Com a estrutura de pastas criada, é possivel fazer o download dos arquivos disponiblizados pelo IBGE. Há uma infinidade de dados.

- https://servicodados.ibge.gov.br/api/docs


<br>

## São Paulo


In [ ]:
# Define qual o código IBGE do Estado
estado = 35

# Define URL
parameters = {
    'formato': 'application/vnd.geo+json',
    'resolucao': '5',
    'qualidade': '4',
}
url = 'https://servicodados.ibge.gov.br/api/v2/malhas/{}'.format(estado)
r = requests.get(url, params=parameters)
print(r.url)

# Define o nome do arquivo que será salvo as informações do IBGE
geojson_file = os.path.join(input_path, 'sp_ibge.geojson')

# Save
with open(geojson_file, 'wb') as f:
    f.write(r.content)

In [ ]:
file_encoding = tt.predict_encoding(geojson_file)
file_encoding

In [ ]:
gdf = gpd.read_file(
    geojson_file    
)

gdf.rename(
    {'codarea': 'id_ibge'},
    axis=1,
    inplace=True
)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Salva Arquivo
gdf.to_file(
    os.path.join(output_path_geo, 'sp_igbe.gpkg'),
    layer='Limite Municipal',
    driver='GPKG',
    encoding='utf-8'
)
gdf

In [ ]:
# Map Object
m = folium.Map()

# Folium Object
folium.GeoJson(geojson_file).add_to(m)

# Fit and Plot map
m.fit_bounds(m.get_bounds())
m

<br>

## São Paulo (Memória)


<br>

Ou ainda, ao invés de baixar o arquivo, é possivel fazer com o que o mapa seja criado com a leitura dos dados diretamente do site do IBGE. Nessa função o _encoding_ já foi definido, evitando o problema mencionado acima.


In [ ]:
r = requests.get(url, params=parameters)
geojson = json.loads(r.text)

In [ ]:
# Map Object
m = folium.Map()

# Folium Object
folium.GeoJson(geojson).add_to(m)

# Fit and Plot map
m.fit_bounds(m.get_bounds())
m

<br>

## Estados


In [ ]:
# Define qual o código IBGE do Estado
estado = 'UF'

# Define URL
parameters = {
    'formato': 'application/vnd.geo+json',
    #'resolucao': '5',
    #'qualidade': '4',
}
url = 'https://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?intrarregiao={}'.format(estado)
r = requests.get(url, params=parameters)
print(r.url)

# Define o nome do arquivo que será salvo as informações do IBGE
geojson_file = os.path.join(input_path, 'br_estados_ibge.geojson')

# Save
with open(geojson_file, 'wb') as f:
    f.write(r.content)

In [ ]:
gdf = gpd.read_file(
    geojson_file    
)

gdf.rename(
    {'codarea': 'id_ibge'},
    axis=1,
    inplace=True
)

# Transforma Coordenadas
gdf = gdf.to_crs(epsg=4326)

# Salva Arquivo
gdf.to_file(
    os.path.join(output_path_geo, 'br_igbe.gpkg'),
    layer='Limite Municipal',
    driver='GPKG',
    encoding='utf-8'
)
gdf.head()

In [ ]:
# API
list_estados = []
for index, row in gdf.iterrows():
    url = 'https://servicodados.ibge.gov.br/api/v1/localidades/estados/{UF}'.format(UF=row['id_ibge'])
    r = requests.get(url)
    dict_quotes = r.content.decode(r.encoding)
    dict_quotes = ast.literal_eval(dict_quotes)
    list_estados.append(dict_quotes)
    
# Transforma dictionary in Dataframe
df = pd.DataFrame(list_estados)
df.head()

# Adjust nested Dictionary
for i, row in df.iterrows():
    df.loc[i, 'id_regiao'] = row['regiao']['id']
    df.loc[i, 'sigla_regiao'] = row['regiao']['sigla']
    df.loc[i, 'nome_regiao'] = row['regiao']['nome']

# Fim
df.drop(['regiao'], axis=1, inplace=True)
df['id_regiao'] = pd.to_numeric(df['id_regiao'], downcast='integer')
df.head()

In [ ]:
df.to_csv(
    os.path.join(output_path_tab, 'tab_uf_ibge.csv'),
    index=False
)

In [ ]:
# Map Object
m = folium.Map()

# Folium Object
folium.GeoJson(geojson_file).add_to(m)

# Fit and Plot map
m.fit_bounds(m.get_bounds())
m

<br>

Uma vez com o mapa na mão, de qualquer que seja o meio que foi obtido, é possivel analisar a "tabela de atributos".
Lá descobrimos que existe o par de coordenadas que define o centroide e, ainda, o 'codarea' que tem o código do IBGE do município.


<br>

# Dados Tabulares


In [ ]:
df_ufs = pd.read_csv(
    os.path.join(output_path_tab, 'tab_uf_ibge.csv'),
)

list_dfs = []
for i, row  in df_ufs.iterrows():
    estado = row['id']
    url = 'http://servicodados.ibge.gov.br/api/v1/localidades/estados/{}/municipios'.format(estado)    
    df = pd.read_json(url)
    list_dfs.append(df)

# Ajustes
df = pd.concat(list_dfs)
df = df[['id', 'nome']].copy()

# Salva
df.to_csv(
    os.path.join(output_path_tab, 'tab_municipio_ibge.csv'),
    index=False
)
df.head()

In [ ]:
# Defines IBGE Code State
estado = 35

# Defines URL
url = 'http://servicodados.ibge.gov.br/api/v1/localidades/estados/{}/municipios'.format(estado)
print('{}\n'.format(url))

# Create Dataframe
df = pd.read_json(url)

# Seleciona Colunas
df = df[['id', 'nome']]

# Renomeia Colunas
df = df.rename(columns=lambda x: x.replace('id', 'id_ibge'))
df

<br>

# Mapa Categórico


In [ ]:
# Adjust dtypes
#df['id_ibge'] = df['id_ibge'].apply(lambda x: str(x))

# Add Random Colum
df['random'] = np.random.uniform(1, 6, df.shape[0])

# Results
print('{}\n'.format(df.dtypes))
display(df.head())

In [ ]:
# Geodataframe
gdf = gpd.read_file(
    os.path.join(output_path_geo, 'sp_igbe.gpkg')
)

# Ajusta os Tipo
gdf['id_ibge'] = pd.to_numeric(gdf['id_ibge'], downcast='integer')

# Results
print('{}\n'.format(gdf.dtypes))
display(gdf.head())

In [ ]:
# Create Map
m = folium.Map()

highlight_function = lambda x: {
    'fillColor': '#000000', 
    'color':'#000000',
    'fillOpacity': 30,
    'weight': 1
}

choropleth = folium.Choropleth(
    geo_data=gdf,
    name='choropleth',    
    data=df,
    columns=['id_ibge', 'random', 'nome'],
    key_on='feature.properties.id_ibge',
    fill_color='YlGnBu',
    fill_opacity=0.5,
    line_opacity=0.1,
    legend_name='Legenda',
    smooth_factor=0,
    show=False,
    overlay=True,
    highlight=True,
    highlight_function=highlight_function,
    nan_fill_color='White',
).add_to(m)

# Add Field in geodataframe
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['id_ibge'], labels=False)
)

# Fit and Plot map
m.fit_bounds(m.get_bounds())
m